<br>
# **구글번역 덧붙이기**
https://github.com/ssut/py-googletrans

<br>
## ** 1 자막 자동번역**
구글 번역으로 작업을 확인하고, 파파고를 뒤에 덧붙이기

In [ ]:
# src  = 'en', 'ko', 'la'
# dest = 'en', 'ko', 'ja'
text = '이번의 파이썬 수업은 장고를 중점으로 KAKAO BOT 을 응용하는 실전내용 입니다'

In [4]:
from googletrans import Translator
translator = Translator()
result     = translator.translate(text, src='ko', dest='en')
print(translator.detect(text).lang)
result.text

ko


'This Python lesson is about the application of KAKAO BOT with emphasis on Django'

<br>
## **2 src 자막 번역기 함수**
외국어 자막을 한글로 자동 변환하기

In [1]:
# Txt 파일 불러오기
txt_result = []
f = open("./Under.srt", 'r')
while True:
    line = f.readline()
    if not line: break
    txt_result.append(line)
f.close()

In [2]:
# 번역에 불필요한 내용 제거
def txt_corpus(txt):
    word_corpus = ['<i>', '</i>', '.', '\n', '-']
    for temp in word_corpus:
        if txt.find(temp) != -1:
            txt = txt.replace(temp, '')
    return txt

In [3]:
# srt 원본을 Dict 객체로 정리 (불필요한 텍스트 제거)
result = []
for no, txt in enumerate(txt_result):
    if txt.find('-->') != -1:           # 시간 정보가 존재할 때
        if len(txt_result[no+2]) < 2:   # 텍스트가 1줄
            text = [txt_corpus(txt_result[no+1])]
            time = txt_result[no].replace('\n', '')
            result.append( [time , text]  )
        elif len(txt_result[no+3]) < 2: # 텍스트가 2줄
            text = [txt_corpus(txt_result[no+1]),
                    txt_corpus(txt_result[no+2])]
            time = txt_result[no].replace('\n', '')
            result.append( [time , text]  )
        elif len(txt_result[no+4]) < 2: # 텍스트가 3줄
            text = [txt_corpus(txt_result[no+1]),
                    txt_corpus(txt_result[no+2]), 
                    txt_corpus(txt_result[no+3])]
            time = txt_result[no].replace('\n', '')
            result.append( [time , text]  )
        else: pass
result[:5]

[['00:01:00,037 --> 00:01:01,410', ['So where do you think', "you're going?"]],
 ['00:01:01,411 --> 00:01:02,881', ['Just for a breath of fresh air']],
 ['00:01:02,882 --> 00:01:03,841', ['Tony']],
 ['00:01:03,842 --> 00:01:05,598', ["He won't be back for hours"]],
 ['00:01:05,599 --> 00:01:06,558', ['Tony!']]]

In [4]:
# 구글번역기 사용자 함수
from googletrans import Translator
import time
def google_trans(txt, sleep=0, src='en', dest='ko'):
    translator = Translator()
    time.sleep(sleep) # API 호출간격을 설정
    txt = translator.translate(txt, src=src, dest=dest)
    txt = txt.text
    return txt

In [ ]:
# SRT 객체를 생성한다 
import time
total_num = len(result)
for no in range(total_num):
    if no % 150 == 0:
        print(no, ":", total_num, ' -- line is Processing Now...')
    line_texts = result[no][1]
    txt_result = []
    for txt in line_texts:
        try:
            #txt_trans = papago_trans(txt, source='en')
            #time.sleep(0.5)
            txt_trans = google_trans(txt)  # 구글번역이 빠르고 반응성이 더 좋다
            txt_result.append(txt_trans)
        except:         # 번역작업에 오류가 발생시
            txt_result.append(txt)
            print(no, 'line is Error by Json')
    result[no][1] = txt_result
print('Translate is Done..!!')

0 : 1478  -- line is Processing Now...
150 : 1478  -- line is Processing Now...
300 : 1478  -- line is Processing Now...
450 : 1478  -- line is Processing Now...
518 line is Error by Json
519 line is Error by Json


In [ ]:
f = open("./result.srt", 'a')
for no in range(len(result)):
    temp = result[no] # 해당 줄 작업시작
    f.write(str(no+1) + '\n')     # 작업번호 입력
    f.write(result[no][0] + '\n') # Time line 입력
    if len(result[no][1]) == 1:
        f.write(result[no][1][0] + '\n') # 대사1줄 저장
    else:
        for txt in result[no][1]:
            if len(txt) < 2: pass
            else:            f.write(txt + '\n') # 대사를 저장
    f.write('\n')     # 마지막 여백추가
f.close()
print('Done!!')

<br>
# **자막번역을 파파고로 자동화 작업하기**

In [21]:
def papago_trans(text, source='ko'):
    # 네이버 기계번역 API 예제 : 1일 만단어 제한
    import os, sys, urllib.request, time, json
    client_id     = "HCix79l7xff3LZ_DR_Re"
    client_secret = "sDD1cOLSql"
    encText       = urllib.parse.quote(text)
    url           = "https://openapi.naver.com/v1/language/translate"   
    if source == 'ko':
        data = "source=ko&target=en&text=" + encText # 한글을 영어로
    elif source == 'en':
        data = "source=en&target=ko&text=" + encText # 영어를 한글로
    else : pass
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        text          = response_body.decode('utf-8') 
        text          = json.loads(text)
        return text['message']['result']['translatedText']
    else:
        return "Error Code:" + rescode + " 다음의 원인으로 번역이 불가능 합니다"    

In [17]:
papago_trans("I didn't think you were serious.")

"I didn't think you were serious."

In [22]:
def myfunction():
    myfunction.counter += 1

myfunction.counter = 0

In [27]:
def multi_ten():
    square_init = 10
    
    def square(x):
        return square_init ** x
    return square

In [28]:
f = multi_ten()
f(2)

100

In [31]:
f.__closure__[0]

<cell at 0x7f7acd505858: int object at 0xa6dde0>

In [37]:
def closure():
    x = 10
    
    def inner():
        y = 20
        return x + y
    
    return inner

In [40]:
p = closure()
p()

30

In [39]:
len(p.__closure__)

1

In [36]:
dir(p.__closure__[0].cell_contents)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

In [69]:
def deco(func):    
    def wrapper():
        print("before --------")
        ret = func()
        print('after  --------')
        return ret
    return wrapper

In [70]:
@deco
def base():
    print("base function")

In [71]:
base()

before --------
base function
after  --------


In [72]:
def base():
    print("base function")

deco(base)()

In [67]:
deco(base)()

before --------
before --------
base function
after  --------
after  --------


In [77]:
# "func"로 감싼 객체를 wrapper()를 추가 실행한다
def measure_run_time(func):
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = func(*args, **kwargs)
        end    = time.time()
        print("{} function running time : {} sec".format(func.__name__, int(end - start)))
        return result
    return wrapper

In [78]:
import time

@measure_run_time
def worker(delay_time):
    time.sleep(delay_time)

In [79]:
worker(4)

worker function running time : 4 sec


<br>
## **2개 이상의 Decorator를 사용**
꼬이는 작업을 피하는 방법

In [83]:
import time, datetime
from functools import wraps

def parameter_logger(func):    
    @wraps(func)
    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        print("[%s] args : %s, kwargs : %s" %(timestamp, args, kwargs))
        return func(*args, **kwargs)
    return wrapper

def measure_run_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print("{} function running time : {} sec".format(func.__name__, int(end - start)))
        return result
    return wrapper

In [88]:
@measure_run_time
@parameter_logger
def worker(delay_time):
    time.sleep(delay_time)

In [90]:
worker(3)

[2018-07-14 15:28] args : (3,), kwargs : {}
worker function running time : 3 sec


<br>
## **3 클래스 객체 내부에서  Decorator 사용**
def &nbsp;&nbsp; \_\_call\_\_ &nbsp;&nbsp; ():

In [91]:
import time
from functools import update_wrapper

class MeasureRuntime:
    # 클래스 생성자 정의
    def __init__(self, f):
        self.func = f
        update_wrapper(self, self.func)
    
    # 클래스를 사용하기 위해 정의할 때 호출 
    def __call__(self, *args, **kwargs):
        start  = time.time()
        result = self.func(*args, **kwargs)
        end    = time.time()
        print("{} function running time : {} sec".format(self.func.__name__, int(end - start)))
        return result

In [98]:
@MeasureRuntime
def worker(delay_time):
    time.sleep(delay_time)
worker(3)

worker function running time : 3 sec


In [99]:
## 단 데코레이터에서도 매개변수를 갖는 경우는 closure 를 써야한다